In [92]:
import requests

In [93]:
from bs4 import BeautifulSoup

In [94]:
from datetime import date

In [95]:
page = requests.get("https://cookpolitical.com/ratings/house-race-ratings")

In [96]:
page

<Response [200]>

In [97]:
soup = BeautifulSoup(page.content, 'html.parser')

In [98]:
type(soup)

bs4.BeautifulSoup

In [99]:
# write soup to html to save when future ratings changes would overwrite
with open("house-"+str(date.today())+".html", "w") as file:
    file.write(str(soup))

In [100]:
# --------- How to read it saved html soup ---------
# with open("house-"+str(date.today())+".html", 'r') as f:
#     contents = f.read()
#     soupread = BeautifulSoup(contents, 'lxml')
# test = soupread.find_all(class_='popup-table-data-row')

In [101]:
categories = soup.find_all(class_='popup-table-data-row')

In [102]:
type(categories)

bs4.element.ResultSet

In [103]:
seats=[]

In [104]:
for cat in categories:
    seats.append([elem.get_text().strip() for elem in cat.select("a")])

In [105]:
seatsList = [[]]
for i in seats:
    if not i:
        seatsList.append([[]])
    else:
        seatsList[-1].append(i)

In [106]:
seatsList = seatsList[1:]

In [107]:
allS = [[]]
for i in seatsList:
    allS.append([x for x in i if x != []])

In [108]:
allS=allS[1:]

In [109]:
allSeats=[]
for i in allS:
    allSeats.append([item for sublist in i for item in sublist])

In [110]:
names=[]

In [111]:
for cat in categories:
    names.append([elem.get_text().strip() for elem in cat.select("div")])

In [112]:
namesList = [[]]
for i in names:
    if i[0]=="DISTRICT":
        namesList.append([[]])
    else:
        namesList[-1].append(i[0])

In [113]:
namesList = namesList[1:]

In [114]:
allNames = [[]]
for i in namesList:
    allNames.append([x for x in i if x != []])

In [115]:
allNames=allNames[1:]

In [116]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [117]:
seatsdf= pd.DataFrame(allSeats).transpose()

In [118]:
cats=['SOLID_DEMOCRAT','LIKELY_DEMOCRAT','LEAN_DEMOCRAT','TOSS_UP_DEM','SOLID_REPUBLICAN','LIKELY_REPUBLICAN','LEAN_REPUBLICAN','TOSS_UP_REP']

In [119]:
seatsdf.columns=cats

In [120]:
seatsdf.head()

,SOLID_DEMOCRAT,LIKELY_DEMOCRAT,LEAN_DEMOCRAT,TOSS_UP_DEM,SOLID_REPUBLICAN,LIKELY_REPUBLICAN,LEAN_REPUBLICAN,TOSS_UP_REP
0,AL-07,CA-10,AZ-01,CA-21,AL-01,CA-04,AK-00,AZ-06
1,AZ-02,CA-39,CA-48,IA-01,AL-02,CA-22,AR-02,CA-25
2,AZ-03,FL-27,FL-26,IA-02,AL-03,FL-16,CA-50,IL-13
3,AZ-07,GA-06,GA-07,MN-07,AL-04,FL-18,CO-03,IN-05
4,AZ-09,IL-14,IA-03,NM-02,AL-05,KS-02,FL-15,MI-03


In [121]:
seatsdf=seatsdf[['SOLID_DEMOCRAT','LIKELY_DEMOCRAT','LEAN_DEMOCRAT','TOSS_UP_DEM','TOSS_UP_REP','LEAN_REPUBLICAN','LIKELY_REPUBLICAN','SOLID_REPUBLICAN']]

In [122]:
hseats = pd.melt(seatsdf).dropna()

In [123]:
namesdf= pd.DataFrame(allNames).transpose()

In [124]:
namesdf.columns=cats

In [125]:
namesdf=namesdf[['SOLID_DEMOCRAT','LIKELY_DEMOCRAT','LEAN_DEMOCRAT','TOSS_UP_DEM','TOSS_UP_REP','LEAN_REPUBLICAN','LIKELY_REPUBLICAN','SOLID_REPUBLICAN']]

In [126]:
hnames = pd.melt(namesdf).dropna()

In [139]:
h= pd.concat([hseats, hnames['value']],axis=1)

In [140]:
h.columns=['rating','seat','name']

In [141]:
h['rating'][(h['rating']=="TOSS_UP_DEM")|(h['rating']=="TOSS_UP_REP")]="TOSS_UP"

In [142]:
held=pd.read_csv('house_held.csv')

In [143]:
h=h.merge(held,on="seat")

In [144]:
house=h.reset_index(drop=True)

In [145]:
house['index']=house.index

In [146]:
house.to_json('house.json',orient='records')

In [147]:
house

,rating,seat,name,held,held_name,index
0,SOLID_DEMOCRAT,AL-07,Terri Sewell,D,Terri Sewell,0
1,SOLID_DEMOCRAT,AZ-02,Ann Kirkpatrick,D,Ann Kirkpatrick,1
2,SOLID_DEMOCRAT,AZ-03,Raúl Grijalva,D,Raúl Grijalva,2
3,SOLID_DEMOCRAT,AZ-07,Ruben Gallego,D,Ruben Gallego,3
4,SOLID_DEMOCRAT,AZ-09,Greg Stanton,D,Greg Stanton,4
5,SOLID_DEMOCRAT,CA-02,Jared Huffman,D,Jared Huffman,5
6,SOLID_DEMOCRAT,CA-03,John Garamendi,D,John Garamendi,6
7,SOLID_DEMOCRAT,CA-05,Mike Thompson,D,Mike Thompson,7
8,SOLID_DEMOCRAT,CA-06,Doris Matsui,D,Doris Matsui,8
9,SOLID_DEMOCRAT,CA-07,Ami Bera,D,Ami Bera,9
